# 1) imports

In [1]:
import findspark
findspark.init()
# Spark:
from pyspark     import SparkConf
from pyspark     import SparkContext
from pyspark.sql import SparkSession

# Python :
from datetime import datetime
import math

# 2) configuration de spark

In [2]:
user           = "romain - Analyse cycliste"
ip_fares       = "54.37.12.49"
master         = "spark://54.37.12.49:7077"
executor_cores = 2 # Ne prendra que des machines qui ont des exécuteurs qui peuvent avoir ce nombre de coeurs
                   # nb de cores par executor (doit être inférieur au nombre de CPU par machine)
nb_cores_max   = 6 # détermine le nombre d'executeur max => max( arrondi.min(nb_cores_max/executor_cores)
parallelism    = 4 * executor_cores
nb_executor    = nb_cores_max//executor_cores
current_date   = datetime.now().strftime("%Y-%m-%d %H:%M")
appName        = "%s le %s"%(user, current_date)
print("il y aura %s executors avec chacun %s coeurs (possiblement sur moins de machines)"%(nb_executor, executor_cores))


il y aura 3 executors avec chacun 2 coeurs (possiblement sur moins de machines)


# 3) création d'un objet "configuration"

In [3]:
dico_conf = {
    "spark.app.name"             : appName        ,
    "spark.master"               : master         ,
    "spark.cores.max"            : nb_cores_max   , 
    "spark.executor.cores"       : executor_cores ,
    "spark.default.parallelism"  : parallelism    , 
    "spark.python.worker.memory" : "1042M"        ,
}
conf = SparkConf()
for k,v in dico_conf.items():
    conf = conf.set(k,v)

# 4) création d'un spark context et d'une spark Session

In [4]:

spark = SparkSession.builder.getOrCreate()


In [5]:
sc = spark.sparkContext

# 5) créons une dataframe : 

In [6]:
file_path = "hdfs://54.37.12.49:8020/cyclistes/cycliste_cyclistes.csv"
file_path = "./cycliste_cyclistes.csv"
df        = spark.read.option("header", "true").\
                    option("inferSchema", "true").\
                    csv(file_path)

# 6) changer les strings en int

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
string_Indexer = StringIndexer()
string_Indexer.setInputCol("sexe")
string_Indexer.setOutputCol("label")

string_indexer_model = string_Indexer.fit(df)
cyclistes_doubles    = string_indexer_model.transform(df)

# 7) création d'une colonne de vecteurs

In [9]:
from pyspark.ml.feature import VectorAssembler

In [12]:
colonnes             = cyclistes_doubles.columns
colonnes_a_supprimer = ["sexe", "sexe-num", "cycliste", "label"]
colonnes_sans_y      = [col for col in colonnes if col not in colonnes_a_supprimer]

In [13]:
x_y = VectorAssembler()
x_y.setInputCols(colonnes_sans_y)
x_y.setOutputCol("features")
points = x_y.transform(cyclistes_doubles)

In [14]:
points.columns

['cycliste',
 'sportif',
 'age',
 'sexe',
 'nb_km',
 'vitesse',
 'attente',
 'label',
 'features']

# 10) création d'un jeu de test et d'apprentissage

In [15]:
splits              = points.randomSplit([0.8, 0.2])
jeu_d_apprentissage = splits[0].cache()
jeu_de_test         = splits[1].cache()

# 11) création d'un algo (cart)

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier
dt          = DecisionTreeClassifier()
dtmodel     = dt.fit(jeu_d_apprentissage)
predictions = dtmodel.transform(jeu_de_test)

In [17]:
jeu_d_apprentissage.take(1)

[Row(cycliste='cycliste_aze10', sportif=2.0, age=77, sexe='femme', nb_km=8.55470669961, vitesse=7.17335280185, attente=0.5, label=0.0, features=DenseVector([2.0, 77.0, 8.5547, 7.1734, 0.5]))]

## 11.1) mesure de qualité

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bceval = BinaryClassificationEvaluator()
bceval.evaluate(predictions)

0.5581855136733185

# 12) enregistrement et loading sur hdfs

In [19]:
output_file = "hdfs://54.37.12.49:8020/cart_model_for_cycliste_v5"
output_file = "./cart_model_for_cycliste_v5"
dtmodel.save(output_file)

In [20]:
from pyspark.ml.classification import DecisionTreeClassificationModel
loaded_model = DecisionTreeClassificationModel.load(output_file)

In [23]:
loaded_model.transform(jeu_de_test).collect()

[Row(cycliste='cycliste_aze18', sportif=4.0, age=39, sexe='homme', nb_km=4.78176322402, vitesse=25.6559959231, attente=0.5, label=1.0, features=DenseVector([4.0, 39.0, 4.7818, 25.656, 0.5]), rawPrediction=DenseVector([9.0, 213.0]), probability=DenseVector([0.0405, 0.9595]), prediction=1.0),
 Row(cycliste='cycliste_aze20', sportif=-0.5, age=57, sexe='femme', nb_km=16.3835991264, vitesse=8.04840863069, attente=0.5, label=0.0, features=DenseVector([-0.5, 57.0, 16.3836, 8.0484, 0.5]), rawPrediction=DenseVector([572.0, 182.0]), probability=DenseVector([0.7586, 0.2414]), prediction=0.0),
 Row(cycliste='cycliste_aze26', sportif=2.0, age=43, sexe='femme', nb_km=8.66442400841, vitesse=21.0025539152, attente=0.5, label=0.0, features=DenseVector([2.0, 43.0, 8.6644, 21.0026, 0.5]), rawPrediction=DenseVector([77.0, 57.0]), probability=DenseVector([0.5746, 0.4254]), prediction=0.0),
 Row(cycliste='cycliste_aze27', sportif=2.0, age=32, sexe='homme', nb_km=4.00846311095, vitesse=30.1141759544, attente